In [1]:
import os
import h5py
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Define the input and output directories
input_dir = r"C:\Users\topohl\Documents\SLEAP\Projects\AnalysisSocP\predictions\geom"
output_dir = r"C:\Users\topohl\Documents\SLEAP\Projects\AnalysisSocP\predictions\geom"

# Define geom indices
geom_indices = {
    0: 'tl',
    1: 'tr',
    2: 'br',
    3: 'bl',
    4: 'socl',
    5: 'socr'
}

def fill_missing(Y, kind="linear"):
    """Fills missing values independently along each dimension after the first."""
    if len(Y) == 0:
        return Y
    else:
        # Store initial shape.
        initial_shape = Y.shape
        # Flatten after first dim.
        Y = Y.reshape((initial_shape[0], -1))
        # Interpolate along each slice.
        for i in range(Y.shape[-1]):
            y = Y[:, i]
            if np.isnan(y).all():
                y[:] = np.nan
            else:
                # Build interpolant.
                x = np.flatnonzero(~np.isnan(y))
                f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)
                # Fill missing
                xq = np.flatnonzero(np.isnan(y))
                y[xq] = f(xq)
                # Fill leading or trailing NaNs with the nearest non-NaN values
                mask = np.isnan(y)
                if mask.any():
                    first, last = mask.argmax(), len(mask) - mask[::-1].argmax()
                    y[:first] = y[first]
                    y[last:] = y[last - 1]
            # Save slice
            Y[:, i] = y
        # Restore to initial shape.
        Y = Y.reshape(initial_shape)
        return Y

# Loop over all files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".h5"):
        # Construct the full input and output file paths
        input_path = os.path.join(input_dir, filename)
        output_filename = os.path.splitext(filename)[0] + '_geom_locs.csv'
        output_path = os.path.join(output_dir, output_filename)

        # Load the data from the input file
        with h5py.File(input_path, "r") as f:
            locations = f["tracks"][:].T

        # Fill missing data in the locations array
        locations = fill_missing(locations)

        # Extract geom locations
        geom_locs = {}
        for index, geom_name in geom_indices.items():
            geom_locs[geom_name] = locations[:, index, :, :]

        # Initialize the DataFrame
        df = pd.DataFrame()

        # Loop through the geoms
        for geom in part_indices.values():
            x = geom_locs[geom][:, 0] # retrieve the x coordinate
            y = geom_locs[geom][:, 1] # retrieve the y coordinate
            x, y = x.ravel(), y.ravel() # unravel the arrays
            df[f"{geom}_x"] = x # add the x coordinate to the DataFrame
            df[f"{geom}_y"] = y # add the y coordinate to the DataFrame

        # Save the DataFrame as a csv file
        df.to_csv(output_path, index=False)

        print(f"Processed file {input_path}")

<class 'NameError'>: name 'locations' is not defined